In [ ]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys
import os
sys.path.insert(0, os.path.abspath('../'))
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from src.utils.load_cfg import ConfigLoader
from src.factories import ModelFactory, LossFactory, InferFactory
from src.loaders.base_loader_factory import BaseDataLoaderFactory
from src.factories import DataAugmentationFactory, DataSamplerFactory, DatasetFactory
from trainer import train
from tester import test
import src.utils.logging as logging
from PIL import Image, ImageDraw
import os.path as osp

In [12]:
data_cfg = "./configs/dataset_cfgs/luna16.yaml"
dataset_name, dataset_params = ConfigLoader.load_dataset_cfg(data_cfg)
data_fact = DatasetFactory()
luna16dataset = data_fact.generate(dataset_name, mode="train", **dataset_params)
luna16loader  = DataLoader(luna16dataset, shuffle=False, drop_last=False) 

In [ ]:
for i, (samples, labels) in enumerate(luna16loader):
    smpl = samples[0]
    lbl  = labels
    lbl  = [c.item() for c in lbl]
    x, y, z, rx, ry, rz = tuple(lbl)
   
    smpl_npy   = smpl.squeeze().detach().numpy()
    slices_idx = [max(0, z-rz),min(smpl.shape[0], z+int(rz))]
    for slidx in range(*slices_idx):
        smpl_uint  = (smpl_npy[slidx] * 255).astype(np.uint8)
        smpl_pil   = Image.fromarray(np.concatenate([smpl_uint[...,np.newaxis]]*3, axis = 2))
    
        fname      = nod_fname_lst[i]
        slc_img_name = str(fname)+"_"+str(slidx)+".jpg"
        slc_img_pth  = osp.join(out_dir,fname, slc_img_name)
        print(slc_img_pth)
        os.makedirs(osp.dirname(slc_img_pth), exist_ok=True)
        smpl_pil.save(slc_img_pth)
    break
        # Place data on the corresponding device

In [ ]:
smpl_drw = ImageDraw.Draw(smpl_pil)
smpl_drw.point([y,x], fill='green')
smpl_drw.rectangle([(x-rx, y-ry),(x+rx,y+ry)], outline = 'green')
smpl_pil